## __THE BATTLE OF NEIGHBORHOODS - PART 1__

### __JAPANESE RESTAURANTS IN NEW YORK CITY__

_SNEHA SRUTHI V_

__INTRODUCTION__

Konnichiwa!!

This final project explores New York City for the best locations of Janese restaurants throughout the city. 
New York is a major metropolitan area with more than 8.8 million (Macrotrends, 2020) people living within city limits. The earliest Japanese immigrants to settle in NYC came in 1876, mostly merchants who set up businesses to import silk. By 1900, there were approximately 1,000 first generation immigrants in New York State, almost all of them in New York City.

As of 2019, 30,000 Japanese nationals and a total of 45,000 reside in the Greater New York City area. Many of the Japanese are from transient groups such as university students, artists, and business workers.
Over 30,000 Japanese nationals live throughout the five boroughs of New York City. However, New York's "Japantown" usually referred to as Little Tokyo, is located on the Lower East Side, between 2nd and 3rd Avenues and 8th (St Mark's Place) and 10th Streets, nothing comparable to New York's famous Chinatown.

This report explores which neighborhood and boroughs of New York City have the most as well as the best Japanese restaurants. Additionally, there will be an attempt to answer the questions “Where should I open an Japanese Restaurant?” and “Where should I stay If I want great Japanese food?” to meet the needs of the working professionals and students. 

__DATA__

-  In order to answer the above questions, data on New York City neighborhoods, boroughs, including boundaries, latitude, longitude, restaurants, and restaurant ratings and tips are required.

-  the above information will be obtained from New York City data containing the neighborhoods and boroughs, latitudes, and longitudes. The data source used will be: __https://cocl.us/new_york_dataset__

-  The neighborhood boundaries of New York City will be obtained from the data source: __https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm__

-  FourSquare API will be utilized via the Request library in Python to procure all data related to locations and user ratings of Japanese restaurants

__METHOD__

Data will be collected from https://cocl.us/new_york_dataset cleaned and processed into a dataframe.

-  FourSquare be used to locate all venues and then filtered by Japanese restaurants. Ratings, tips, and likes by users will be counted and added to the dataframe.

-  Data will be sorted based on rankings

-  Finally, the data be will be visually assessed using graphing from various Python libraries like Folium

_For Example, the top 5 neighborhoods in New York City will be selected and the Foursquare API will be called to analyze total number of Japanese restaurants across these neighborhoods_

__Sno      Borough       Neighborhood    Latitude     Longitude__
<br>
301       Manhattan     Hudson Yards    40.756658    -74.000111
<br>
<br>
302        Queens         Hammels       40.587338    -73.805530
<br>
<br>
303        Queens         Bayswater     40.611322    -73.765968
<br>
<br>
304        Queens       Queensbridge    40.756091    -73.945631
<br>
<br>
305    Staten Island      Fox Hills     40.617311    -74.081740

_Below are the Japanese restaurants in the top 5 neighborhoods_

__Borough    Neighborhood    Name__
<br>
<br>
Manhattan    Tudor City  Tempura Matsui
<br>
<br>
Manhattan     Flatiron     Sugarfish
<br>
<br>
Manhattan     Flatiron        HALL
<br>
<br>
Queens    Sunnyside Gardens  Oki Japanese Restaurant
<br>
<br>
Brooklyn    Fulton Ferry     Bessou

__PROBLEM STATEMENT__

-  What is / are the best location(s) for Japanese cuisine in New York City?

-  In what Neighborhood and/or borough should I open a Japanese restaurant to have the best chance of being successful?

-  Where would I go in New York City to have the best Japanese food?